In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import ast
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import re

DATA STEAM

In [2]:
# Ruta del archivo .json.gz
ruta_del_archivo = 'G:\PI MLOps - STEAM\steam_games.json.gz'
# Leemos el archivo JSON comprimido y cargamos los datos en un DataFrame
data = pd.read_json(ruta_del_archivo, compression='gzip', lines=True)
# Especificamos la ruta donde deseas guardar el archivo CSV
ruta_csv = 'G:\PI MLOps - STEAM\steam_games.csv'
# Guardamos el DataFrame como un archivo CSV
data.to_csv(ruta_csv, index=False)

In [3]:
data_steam = pd.read_csv('G:\PI MLOps - STEAM\steam_games.csv')
# Borramos filas vacias
data_steam = data_steam.dropna()

C:\Users\pakin\AppData\Local\Temp\ipykernel_10460\2129812023.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data_steam = pd.read_csv('G:\PI MLOps - STEAM\steam_games.csv')


In [4]:
# Eliminamos todos los valores NaN
data_steam.dropna(how='all', inplace=True)
# Asigamos 0 a todos los registros sin 'id'
data_steam['id'] = data_steam['id'].fillna(0).astype(int)
# Convertimos a tipo INT todos los 'id'
data_steam['id'] = data_steam['id'].astype(int)

In [5]:
# Convertimos las fechas a tipo DateTime y extraemos el año
data_steam['release_date'] = pd.to_datetime(data_steam['release_date'], errors='coerce')
data_steam['release_date'] = data_steam['release_date'].dt.year
data_steam['release_date'] = data_steam['release_date'].fillna(0.0)
data_steam['release_date'] = data_steam['release_date'].astype(int)

In [6]:
# Eliminamos las columnas que no utilizaremos
data_steam.drop(columns=['publisher','title','url','reviews_url','specs','price','early_access','developer'], inplace=True)
# Asignamos 'sin dato' a cada columna que contenga un vacio
#data_steam.fillna('Sin dato', inplace=True)

In [7]:
data_steam.head()

,genres,app_name,release_date,tags,id
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",643980
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017,"['Free to Play', 'Simulation', 'Sports', 'Casu...",670290
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017,"['Action', 'Adventure', 'Casual']",767400
88315,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018,"['Action', 'Adventure', 'Simulation', 'FPS', '...",772540


In [30]:
# Guardar el DataFrame en un archivo CSV
data_steam.to_csv('G:\PI MLOps - STEAM\steam_games_ok.csv', index=False)

DATA SET USER_REVIEWS

In [11]:
# Ruta al archivo JSON
archivo = 'G:\\PI MLOps - STEAM\\australian_user_reviews.json'
# Lista para almacenar objetos evaluados desde el archivo
lista = []
# Abrimos el archivo en modo lectura con codificación utf-8
with open(archivo, encoding='utf-8') as file:
    # Leemos cada línea del archivo
    for line in file.readlines():
        # Utilizamos ast.literal_eval para evaluar cada línea como una expresión literal de Python
        # ast.literal_eval evalúa la línea como una expresión literal de Python de forma segura
        lista.append(ast.literal_eval(line))

In [12]:
# Convertimos la lista de diccionarios a un DataFrame
data_user_reviews = pd.DataFrame(lista)
data_user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [ ]:
# Creamos una funcion para la nueva columna sentiment_analysis

# Descargamos el léxico 'vader_lexicon' necesario para SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Función para el análisis de sentimiento
def sentiment_analysis(review):
    # Verificar si 'review' es una lista no vacía
    if isinstance(review, list) and len(review) > 0:
        # Obtener el texto de la revisión desde la primera entrada de la lista
        text = review[0].get('review', '')          
        # Crear una instancia de SentimentIntensityAnalyzer
        sia = SentimentIntensityAnalyzer()        
        # Calcular los puntajes de polaridad usando SentimentIntensityAnalyzer
        sentiment = sia.polarity_scores(text)
        # Extraer el puntaje compuesto (compound score)
        compound_score = sentiment['compound']
        # Determinar la etiqueta de sentimiento basada en el puntaje compuesto
        if compound_score < -0.2:
            return 0  # Sentimiento negativo
        elif compound_score >= -0.2 and compound_score <= 0.2:
            return 1  # Sentimiento neutro
        else:
            return 2  # Sentimiento positivo
    else:
        # Si 'review' no es una lista válida, devolver 1 (sentimiento neutro por defecto)
        return 1

In [14]:
# Aplicamos sentiment_analysis a userReviews_df
data_user_reviews['sentiment_analysis'] = data_user_reviews['reviews'].apply(sentiment_analysis)

In [15]:
# Revisamos el contenido de userReviews_df
data_user_reviews.head(10)

,user_id,user_url,reviews,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",2
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",2
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...",2
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...",2
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...",1
5,Wackky,http://steamcommunity.com/id/Wackky,"[{'funny': '', 'posted': 'Posted May 5, 2014.'...",1
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,"[{'funny': '1 person found this review funny',...",1
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,"[{'funny': '', 'posted': 'Posted July 24.', 'l...",2
8,76561198089393905,http://steamcommunity.com/profiles/76561198089...,"[{'funny': '5 people found this review funny',...",0
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l...",2


In [16]:
# Desanidamos en la lista lista_reviews todos los datos de data_user_reviews

lista_reviews = []
for indice, row in data_user_reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    sentiment_analysis_value = row['sentiment_analysis']  
    reviews = row['reviews']

    for i in reviews:
            review_data = {
                'user_id': user_id,
                'user_url': user_url,
                'sentiment_analysis': sentiment_analysis_value,
                'posted': i.get('posted', ''),
                'item_id': i.get('item_id', ''),
                'recommend': i.get('recommend', ''),  
            }
            lista_reviews.append(review_data) # Agregamos a lista_reviews el diccionario review_data
data_user_reviews_ = pd.DataFrame(lista_reviews)

In [17]:
# Transformamos la columna 'recommend'
data_user_reviews_['recommend'] = data_user_reviews_['recommend'].replace({True:1, False:0})
# Eliminamos duplicados
data_user_reviews_=data_user_reviews_.drop_duplicates(subset=['user_id'])

In [18]:
# Eliminamos las columnas que no utilizaremos 
data_user_reviews_.drop(columns=['user_url',], inplace=True) 

In [19]:
# Función para extraer el año
def extraer_año(fecha):
    # Utilizamos una expresión regular para encontrar el año en el texto
    matches = re.search(r'\b\d{4}\b', fecha)    
    # Devolver el año si se encuentra, o None si no hay coincidencia
    return matches.group() if matches else None

# Aplicar la función a la columna 'columna_fecha'
data_user_reviews_['año_posted'] = data_user_reviews_['posted'].apply(extraer_año)
# Eliminamos las columnas que no utilizaremos 
data_user_reviews_.drop(columns=['posted',], inplace=True) 

In [20]:
data_user_reviews_.head()

,user_id,sentiment_analysis,item_id,recommend,año_posted
0,76561197970982479,2,1250,1,2011
3,js41637,2,251610,1,2014
6,evcentric,2,248820,1,None
12,doctr,2,250320,1,2013
18,maplemage,1,211420,1,2014


In [21]:
# Eliminamos duplicados
data_user_reviews_=data_user_reviews_.drop_duplicates()
# Asignamos 'sin dato' a cada columna que contenga un vacio
data_user_reviews_.fillna('Sin dato', inplace=True)
# Eliminamos todos los valores NaN
data_user_reviews_.dropna(how='all', inplace=True)
# Asigamos 0 a todos los registros sin 'id'
data_user_reviews_['item_id'] = data_user_reviews_['item_id'].fillna(0).astype(int)
# Convertimos a tipo INT todos los 'id'
data_user_reviews_['item_id'] = data_user_reviews_['item_id'].astype(int)

In [22]:
data_user_reviews_.head()

,user_id,sentiment_analysis,item_id,recommend,año_posted
0,76561197970982479,2,1250,1,2011
3,js41637,2,251610,1,2014
6,evcentric,2,248820,1,Sin dato
12,doctr,2,250320,1,2013
18,maplemage,1,211420,1,2014


In [23]:
# Guardar el DataFrame en un archivo CSV
data_user_reviews_.to_csv('G:\\PI MLOps - STEAM\\australian_user_reviews_ok.csv', index=False)

DATASET USER ITEMS

In [9]:
# Ruta al archivo JSON
archivo = 'G:\\PI MLOps - STEAM\\australian_users_items.json'
# Lista para almacenar objetos evaluados desde el archivo
lista = []
# Abrimos el archivo en modo lectura con codificación utf-8
with open(archivo, encoding='utf-8') as file:
    # Leemos cada línea del archivo
    for line in file.readlines():
        # Utilizamos ast.literal_eval para evaluar cada línea como una expresión literal de Python
        # ast.literal_eval evalúa la línea como una expresión literal de Python de forma segura
        lista.append(ast.literal_eval(line))

In [10]:
# Convertimos la lista de diccionarios a un DataFrame
data_user_items = pd.DataFrame(lista)
# Mostrarmos el DataFrame resultante
data_user_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [11]:
# Desanidamos en la lista lista_items todos los datos de data_user_items
lista_items = []
for indice, row in data_user_items.iterrows():
    user_id = row['user_id']
    steam_id = row['steam_id']  
    user_url = row['user_url']
    items = row['items']

    for i in row['items']:
        item_new= {
                'user_id': user_id,
                'steam_id': steam_id,
                'user_url': user_url,
                'item_id': i['item_id'],
                'item_name': i['item_name'],
                'playtime_forever': i['playtime_forever'],
                'playtime_2weeks': i['playtime_2weeks']
        }
        lista_items.append(item_new)
data_user_items_ = pd.DataFrame(lista_items)

In [12]:
data_user_items_.head()

,user_id,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


In [13]:
# Eliminamos las columnas que no utilizaremos
data_user_items_.drop(columns=['steam_id','user_url','playtime_2weeks'],inplace= True)

In [14]:
# Eliminamos duplicados
data_user_items_=data_user_items_.drop_duplicates()
# Asignamos 'sin dato' a cada columna que contenga un vacio
# data_user_items_.fillna('Sin dato', inplace=True)
# Eliminamos todos los valores NaN
data_user_items_.dropna(how='all', inplace=True)
# Asigamos 0 a todos los registros sin 'id'
data_user_items_['item_id'] = data_user_items_['item_id'].fillna(0).astype(int)
# Convertimos a tipo INT todos los 'id'
data_user_items_['item_id'] = data_user_items_['item_id'].astype(int)

In [15]:
data_user_items_.head() #DataFrame para PlayTimeGenre

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0


In [16]:
# Guardar el DataFrame en un archivo CSV
data_user_items_.to_csv('G:\\PI MLOps - STEAM\\data_users_items_ok.csv', index=False)

agrego columna a data_steam

In [17]:
#Realizamos un mapeo para agregar la columna playtieme_forever al data frame steamGames 
mapeo = data_user_items_.set_index('item_id')['playtime_forever'].to_dict()

data_steam['playtime_forever'] = data_steam['id'].map(mapeo)
data_steam.fillna(0, inplace=True)
data_steam['playtime_forever'] = data_steam['playtime_forever'].astype(int)

In [18]:
# Guardar el DataFrame en un archivo CSV
data_steam.to_csv('G:\PI MLOps - STEAM\steam_games_ok.csv', index=False)